In [2]:
def colourIn2(v, colours, Nbeads, Nchains):
    
    met = 0.35
    rad = 0.8
    
    ncols = len(colours)
    
    sel=list()
    for icol in range(ncols):
        sel.append(list())
    
    # Create lists for each colour
    for ichain in range(0, Nchains):
        icol = ichain%ncols
        #print("Chain ",ichain," coloured ", icol)
        for ibead in range(Nbeads):
            iatom = ichain*Nbeads + ibead
            sel[icol].append(iatom)
                
    v.view.clear_representations()
    v.view.add_representation('unitcell', color='#000000')
    
    for icol in range(ncols):
        v.view.add_representation('ball+stick', selection=sel[icol], color=colours[icol], radius=rad, metalness=met)
        
    return v

In [3]:
import numpy as np  # Numpy
import math as m    # Maths

import hs_alkane.alkane as mdl   # Fortran library we'll used to model the alkane chain

In [4]:
Nbeads  = 4    # Butane
Nchains = 250  # 250 chains in chain.xmol

# Initialise the simulation box and alkane module ready to hold chains
mdl.box_initialise()             
mdl.alkane_set_nbeads(Nbeads)    
mdl.alkane_set_nchains(Nchains)  
mdl.alkane_initialise()          

# Read from chain.xmol
mdl.io_read_xmol()               # Read from chain.xmol

In [5]:
from ase import Atoms

# Create and populate ASE object
model_positions = np.empty([Nchains*Nbeads, 3])
ibox = 1
cell_vectors = mdl.box_get_cell(ibox)
displace = 0.5*(cell_vectors[0]+cell_vectors[1]+cell_vectors[2])

for ichain in range(0, Nchains):
    model_positions[Nbeads*ichain:Nbeads*ichain+Nbeads] = mdl.alkane_get_chain(ichain+1, ibox)

for iatom in range(0,Nbeads*Nchains):
    model_positions[iatom] = model_positions[iatom] + displace
    

box_config = Atoms('C1000', positions=model_positions*(1.5/0.4), pbc=True, cell=cell_vectors*(1.5/0.4))

In [6]:
from ase.visualize import view

v = view(box_config, viewer='ngl')
colours = ['#FF1111','#FFAAAA', '#DDDDDD', '#1111FF', '#AAAAFF']
v = colourIn2(v, colours, Nbeads, Nchains )
v